### Importing JSON

In [15]:
import pandas as pd

data = pd.read_json('reviews.json')
data.head()

,comments
0,Aplicativo melhorou nós últimos meses.
1,Ótimo
2,Ótimo e muito prático 🙂🙂
3,"Aplicativo TOP, Fácil, e Prático."
4,estou adorando muito seguro


### Cleaning text

In [16]:
import re
import spacy
import unicodedata

def setup_abbr():
    file = open("abbr_portuguese.txt", encoding='utf-8')
    abbr_dict = {}

    for line in file:
        w = line.split(";")
        abbr_dict[w[0]] = w[1].replace("\n", "")
    file.close()

    return abbr_dict

def clean(data):
    doc = nlp(data)
    doc_lower = doc.text.lower()
    doc_punctuation = u"".join([c for c in unicodedata.normalize('NFKD', doc_lower) if not unicodedata.combining(c)])
    doc_corrected = nlp(" ".join([abbr_dict.get(w, w) for w in doc_punctuation.split()]))
    
    return doc_corrected.text

nlp = spacy.load('pt_core_news_lg')
abbr_dict = setup_abbr()

data['cleaned_reviews'] = data['comments'].apply(clean)
data.head()


,comments,cleaned_reviews
0,Aplicativo melhorou nós últimos meses.,aplicativo melhorou nos ultimos meses.
1,Ótimo,otimo
2,Ótimo e muito prático 🙂🙂,otimo e muito pratico 🙂🙂
3,"Aplicativo TOP, Fácil, e Prático.","aplicativo top, facil, e pratico."
4,estou adorando muito seguro,estou adorando muito seguro


### Stopwords remove

In [17]:
import joblib as jb
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

def token_stop_pos(text):
    tagger = jb.load('POS_tagger_brill.pkl')
    # tags = pos_tag(word_tokenize(text))
    tags = tagger.tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('portuguese')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

data['pos_tagged'] = data['cleaned_reviews'].apply(token_stop_pos)
data.head()

[nltk_data] Downloading package punkt to /home/moabsouza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/moabsouza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/moabsouza/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,comments,cleaned_reviews,pos_tagged
0,Aplicativo melhorou nós últimos meses.,aplicativo melhorou nos ultimos meses.,"[(aplicativo, n), (melhorou, v), (ultimos, Non..."
1,Ótimo,otimo,"[(otimo, None)]"
2,Ótimo e muito prático 🙂🙂,otimo e muito pratico 🙂🙂,"[(otimo, None), (pratico, v), (🙂🙂, n)]"
3,"Aplicativo TOP, Fácil, e Prático.","aplicativo top, facil, e pratico.","[(aplicativo, n), (top, None), (,, None), (fac..."
4,estou adorando muito seguro,estou adorando muito seguro,"[(adorando, v), (seguro, n)]"


### Lemmatizer

In [18]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos: 
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:  
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

data['lemma'] = data['pos_tagged'].apply(lemmatize)
data.head()

,comments,cleaned_reviews,pos_tagged,lemma
0,Aplicativo melhorou nós últimos meses.,aplicativo melhorou nos ultimos meses.,"[(aplicativo, n), (melhorou, v), (ultimos, Non...",aplicativo melhorou ultimos me .
1,Ótimo,otimo,"[(otimo, None)]",otimo
2,Ótimo e muito prático 🙂🙂,otimo e muito pratico 🙂🙂,"[(otimo, None), (pratico, v), (🙂🙂, n)]",otimo pratico 🙂🙂
3,"Aplicativo TOP, Fácil, e Prático.","aplicativo top, facil, e pratico.","[(aplicativo, n), (top, None), (,, None), (fac...","aplicativo top , facil , pratico ."
4,estou adorando muito seguro,estou adorando muito seguro,"[(adorando, v), (seguro, n)]",adorando seguro


### English Translation

In [ ]:
from deep_translator import GoogleTranslator

def translate(text):
    return GoogleTranslator(source='auto', target='en').translate(text)

data['translation'] = data['lemma'].apply(translate)
data.head()

### Information Extraction

In [19]:
import spacy
from spacy.matcher import PhraseMatcher

from spacy import displacy 
import visualise_spacy_tree
from IPython.display import Image, display

nlp = spacy.load('pt_core_news_lg')

In [23]:
def find_names(text):
    
    names = []
    
    # Create a spacy doc
    doc = nlp(text)
    
    #create the list of words to match
    security_terms = ['segur']

    #obtain doc object for each word in the list and store it in a list
    patterns = [nlp(term) for term in security_terms]
                
    # Matcher class object 
    matcher = PhraseMatcher(nlp.vocab) 
    matcher.add("SECURITY_PATTERN", patterns)

    matches = matcher(doc)

    # Finding patterns in the text
    for i in range(0,len(matches)):
        
        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        names.append(str(token))
            
    return names

# Apply function
data['extraction'] = data['lemma'].apply(find_names)

In [24]:
for i in range(len(data)):
    if len(data.loc[i,'extraction'])!=0:
        print(data.loc[i,'lemma'])

  adorando seguro
  agio pratico seguro simplificar vida apenas alguns toque parabens ! ! ! !
  aplicativo bastante seguro pratico . gostando
  bom seguro !
  bom seguro melhor podemos pagar contas sair casa


### Text Blob - Sentiment Analysis

In [ ]:
from textblob import TextBlob

# function to calculate subjectivity 
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity

# function to calculate polarity
def getPolarity(review):
    return TextBlob(review).sentiment.polarity

# function to analyze the reviews
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
analysis_data = pd.DataFrame(data[['comments', 'translation']])

In [ ]:
analysis_data['Subjectivity'] = analysis_data['translation'].apply(getSubjectivity) 
analysis_data['Polarity'] = analysis_data['translation'].apply(getPolarity) 
analysis_data['Analysis'] = analysis_data['Polarity'].apply(analysis)
analysis_data.head()